<a href="https://colab.research.google.com/github/pshreyareddy/CourseProject/blob/main/ProjectCodeForBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sarcasm Classification In Tweets



In [1]:
# Install TensorFlow packages
!pip install tensorflow==2.1.0
!pip install tensorflow-hub
!pip install tensorflow-addons
!pip install sentencepiece

     |████████████████████████████████| 421.8MB 22kB/s 
     |████████████████████████████████| 3.9MB 37.2MB/s 
     |████████████████████████████████| 450kB 38.6MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=3c52cfededb30f8879369517e31cab2f39fd889482414d8ae0c759db5c81bd4a
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [35]:
!pip install ipython-autotime
 
%load_ext autotime

time: 125 µs


Import Required Packages 

In [36]:
import tensorflow as tf; 

print(tf.__version__)

2.1.0
time: 2.45 ms


In [37]:
!python --version

Python 3.6.9
time: 161 ms


In [38]:
import pandas as pd
import numpy as np
import re
import string
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Import tensorflow packages
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,concatenate,Dense,GlobalAveragePooling1D,Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_addons as tfa


time: 5.05 ms


Read Train Data Set

In [39]:

train = pd.read_json('https://raw.githubusercontent.com/CS410Fall2020/ClassificationCompetition/main/data/train.jsonl',lines=True)


time: 417 ms


Check train data 

In [40]:
train.head()

,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


time: 19.5 ms


In [41]:
train.dtypes

label       object
response    object
context     object
dtype: object

time: 5.99 ms


Clean the response column in train dataset

In [42]:
#Removing @USER
train['response']=train['response'].str.replace('@USER', "") 
#Removing <URL>
train['response']=train['response'].str.replace('<URL>', "") 
#Removing 1ormore digits(range 0-9)
train['response']=train['response'].str.replace('\d+', '')
#Converting to lower case
train['response']=train['response'].str.lower()

train['response']=train['response'].str.replace('[^\w\s]','')

time: 74.9 ms


Check the train data after changes

In [43]:
train['response'][0]

'   i dont get this  obviously you do care or you wouldve moved right along  instead you decided to care and troll her '

time: 7.59 ms


In [44]:
train['response'].head()

0       i dont get this  obviously you do care or y...
1      trying to protest about  talking about him a...
2       he makes an insane about of money from the ...
3      meanwhile trump wont even release his sat sc...
4      pretty sure the antilincoln crowd claimed th...
Name: response, dtype: object

time: 9.57 ms


Convert context(list) into a string and apply the same cleaning logic added on response column

In [45]:
train['context']=train['context'].apply(lambda x: ','.join(map(str, x)))
train['context']=train['context'].str.replace('@USER', "") 
train['context']=train['context'].str.replace('<URL>', "") 
train['context']=train['context'].str.lower()
train['context']=train['context'].str.replace('[^\w\s]','')
train['context']=train['context'].str.replace('\d+', '')


time: 189 ms


Check the train data after changes

In [46]:
train['context'][0]

'a minor child deserves privacy and should be kept out of politics  pamela karlan  you should be ashamed of your very angry and obviously biased public pandering  and using a child to do it  if your child isnt named barron  bebest melania couldnt care less  fact  '

time: 3.47 ms


In [47]:
train['context'].head()

0    a minor child deserves privacy and should be k...
1      why is he a loser  hes just a press secretar...
2    donald j  trump is guilty as charged  the evid...
3    jamie raskin tanked doug collins  collins look...
4    man  y  all gone  both sides  the apocalypse o...
Name: context, dtype: object

time: 5.3 ms


Check the length of train data set

In [48]:
len(train)

5000

time: 4.09 ms


Check the class count of target variable 'label' (Balanced dataset with equal composition)

In [49]:
from collections import Counter
Counter(train['label'])

Counter({'NOT_SARCASM': 2500, 'SARCASM': 2500})

time: 6.19 ms


Label encode class variable to 0 and 1 from  NOT_SARCASM and SARCASM

In [50]:
le=LabelEncoder()
train_label=le.fit_transform(train['label'])

time: 2.52 ms


Read test data set and check the first 5 rows 

In [51]:

test = pd.read_json('https://raw.githubusercontent.com/CS410Fall2020/ClassificationCompetition/main/data/test.jsonl',lines=True)

test.head()

,id,response,context
0,twitter_1,"@USER @USER @USER My 3 year old , that just fi...","[Well now that ’ s problematic AF <URL>, @USER..."
1,twitter_2,@USER @USER How many verifiable lies has he to...,[Last week the Fake News said that a section o...
2,twitter_3,@USER @USER @USER Maybe Docs just a scrub of a...,[@USER Let ’ s Aplaud Brett When he deserves i...
3,twitter_4,@USER @USER is just a cover up for the real ha...,[Women generally hate this president . What's ...
4,twitter_5,@USER @USER @USER The irony being that he even...,"[Dear media Remoaners , you excitedly sharing ..."


time: 300 ms


In [52]:
test.dtypes

id          object
response    object
context     object
dtype: object

time: 5.66 ms


Apply the same cleaning logic for response and context columns in test  as done earlier on train data set

In [53]:
test['response']=test['response'].str.replace('@USER', "") 
test['response']=test['response'].str.replace('<URL>', "") 
test['response']=test['response'].str.replace('\d+', '')
test['response']=test['response'].str.lower()
test['response']=test['response'].str.replace('[^\w\s]','')
test['context']=test['context'].apply(lambda x: ','.join(map(str, x)))
test['context']=test['context'].str.replace('@USER', "") 
test['context']=test['context'].str.replace('<URL>', "") 
test['context']=test['context'].str.lower()
test['context']=test['context'].str.replace('[^\w\s]','')
test['context']=test['context'].str.replace('\d+', '')


time: 98.5 ms


Check the test rows after changes

In [54]:
test['response'][0]

'   my  year old  that just finished reading nietzsche and then asked me   ayo papa why these people always trying to cancel someone on twitter  trying to pretend like that makes them better themselves    to which i replied  idk   and he just  cuz hoes mad   im so proud  '

time: 3.27 ms


In [55]:
test['context'][0]

'well now that  s problematic af   my  year old  asked me why they are making fun of native americans    i will take shit that didnt happen for     no  he actually in the gifted program and reads on second grade level    and he knows kansas city is in missouri'

time: 3.56 ms


In [56]:
test.head()

,id,response,context
0,twitter_1,my year old that just finished reading ni...,well now that s problematic af my year old...
1,twitter_2,how many verifiable lies has he told now d...,last week the fake news said that a section of...
2,twitter_3,maybe docs just a scrub of a coach i mean ...,let s aplaud brett when he deserves it he co...
3,twitter_4,is just a cover up for the real hate inside ...,women generally hate this president whats up ...
4,twitter_5,the irony being that he even has to ask why,dear media remoaners you excitedly sharing cl...


time: 21.2 ms


Check the train rows after changes 

In [57]:
train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,SARCASM,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,SARCASM,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,SARCASM,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


time: 20.2 ms


Create a variable traindata from train 

In [58]:
traindata = train

time: 1.01 ms


In [59]:
len(train_label)

5000

time: 5.53 ms


Train Validation Split (To train on 4000 samples and validate on 1000 samples)

In [60]:
X_train, X_val, y_train, y_val = train_test_split(traindata.drop(labels=['label'], axis=1), train_label, test_size=0.2, random_state=42 ,stratify=train_label)
#


time: 15.1 ms


In [61]:
X_train.shape   #Check the shape of X_train

(4000, 2)

time: 5.27 ms


In [62]:
y_train.shape  #Check the shape of y_train


(4000,)

time: 3.75 ms


In [63]:
 X_val.shape   #Check the shape of X_val

(1000, 2)

time: 2.93 ms


Check the target variable balance in training and validation 

In [64]:
print(Counter(y_val))  
print(Counter(y_train))

Counter({0: 500, 1: 500})
Counter({1: 2000, 0: 2000})
time: 3.52 ms


Added a keras bert layer using bert uncased L-12_H-768_A-12. Tokenization approach using bert fulltokenizer followed as per usage in below tfhub link

https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1

In [65]:
import tensorflow_hub as hub
hub

<module 'tensorflow_hub' from '/usr/local/lib/python3.6/dist-packages/tensorflow_hub/__init__.py'>

time: 4.03 ms


In [67]:
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"

bert_layer = hub.KerasLayer(module_url, trainable=True)

time: 3min 15s



Download and import bert tokenization file

In [68]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

time: 378 ms


In [69]:
import tokenization
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

time: 596 ms


Encoding function to separate the text into tokens,masks and segments

In [71]:
# Reference link for encoder function:
# https://www.analyticsvidhya.com/blog/2020/10/simple-text-multi-classification-task-using-keras-bert/
def encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

time: 14.1 ms


In [72]:
 max_len=256

time: 723 µs


In [73]:
val_response = encode(X_val.response, tokenizer, max_len=max_len)
val_context = encode(X_val.context, tokenizer, max_len=max_len)
val_labels=y_val

time: 2.02 s


In [74]:
train_response = encode(X_train.response, tokenizer, max_len=max_len)
train_context = encode(X_train.context, tokenizer, max_len=max_len)
train_labels=y_train


time: 7.71 s


In [75]:
test_response = encode(test.response, tokenizer, max_len=max_len)
test_context = encode(test.context, tokenizer, max_len=max_len)

time: 3.24 s


Set test_input 

In [76]:
test_input=[test_context[0],test_context[1],test_context[2],test_response[0],test_response[1],test_response[2]]

time: 1.47 ms


Helper functions for Evaluation metrics like recall,precision and f1 score from
https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

In [77]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

time: 7.75 ms


Build Model



In [78]:
def build_model(bert_layer, max_len=512):
    
    
    # input word ids,masks,segment ids for context
    # Refered to https://www.analyticsvidhya.com/blog/2020/10/simple-text-multi-classification-task-using-keras-bert/
    context_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="context_word_ids")
    context_mask = Input(shape=(max_len,), dtype=tf.int32, name="context_mask")
    context_seg_ids = Input(shape=(max_len,), dtype=tf.int32, name="context_seg_ids")
    context_pooled_output , context_seq_output = bert_layer([context_word_ids, context_mask, context_seg_ids])
    context_clf_output = context_seq_output[:, 0, :]

    # input word ids,masks,segment ids for response
    reponse_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_word_ids")
    reponse_mask = Input(shape=(max_len,), dtype=tf.int32, name="reponse_mask")
    reponse_seg_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_seg_ids")
    response_pooled_output , reponse_seq_output = bert_layer([reponse_word_ids, reponse_mask, reponse_seg_ids])
    reponse_clf_output = reponse_seq_output[:, 0, :]

    #concatenation layer of outputs of bert layer for context and response
    conclayer=concatenate([context_clf_output,reponse_clf_output])
    # give conclayer as input to a simple dense network with 1 neuron with activation function sigmoid
    out = Dense(1, activation='sigmoid')(conclayer)
    
    #set keras model with inputs as an array of context and response input word ids,mask and segment ids
    model = Model(inputs=[context_word_ids, context_mask, context_seg_ids,reponse_word_ids,reponse_mask,reponse_seg_ids], outputs=out)
    
    #Model compilation
    #Used adam optimizer with learning rate 1e-6, used binary cross entropy loss as its a binary classification problem
    #Calculating accuracy,f1,precision and recall metrics for each run
    model.compile(Adam(1e-6), loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

    return model



time: 20 ms


Check summary of the model

In [79]:

model = build_model(bert_layer, max_len=max_len)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
context_word_ids (InputLayer)   [(None, 256)]        0                                            
__________________________________________________________________________________________________
context_mask (InputLayer)       [(None, 256)]        0                                            
__________________________________________________________________________________________________
context_seg_ids (InputLayer)    [(None, 256)]        0                                            
__________________________________________________________________________________________________
reponse_word_ids (InputLayer)   [(None, 256)]        0                                            
______________________________________________________________________________________________

Set train_input (Array of train_context and train_reponse with inturn contains input word ids,mask and segment ids )

In [80]:
train_input=[train_context[0],train_context[1],train_context[2],train_response[0],train_response[1],train_response[2]]

time: 1.06 ms



Set val_input (Array of val_context and val_reponse with inturn contains input word ids,mask and segment ids )

In [81]:
val_input=[val_context[0],val_context[1],val_context[2],val_response[0],val_response[1],val_response[2]]

time: 1.05 ms


Training the model 

In [82]:
train_history = model.fit(
    train_input, y_train,
    validation_data =(val_input,y_val),
    epochs=3,
    batch_size=3)

Train on 4000 samples, validate on 1000 samples
Epoch 1/3
4000/4000 [==============================] - 346s 87ms/sample - loss: 0.5775 - acc: 0.6835 - f1_m: 0.5654 - precision_m: 0.5852 - recall_m: 0.6027 - val_loss: 0.5129 - val_acc: 0.7520 - val_f1_m: 0.6804 - val_precision_m: 0.6467 - val_recall_m: 0.7725
Epoch 2/3
4000/4000 [==============================] - 317s 79ms/sample - loss: 0.4321 - acc: 0.7968 - f1_m: 0.6888 - precision_m: 0.6932 - recall_m: 0.7341 - val_loss: 0.4604 - val_acc: 0.7860 - val_f1_m: 0.6856 - val_precision_m: 0.6707 - val_recall_m: 0.7495
Epoch 3/3
4000/4000 [==============================] - 317s 79ms/sample - loss: 0.3342 - acc: 0.8575 - f1_m: 0.7505 - precision_m: 0.7527 - recall_m: 0.7834 - val_loss: 0.4556 - val_acc: 0.7930 - val_f1_m: 0.6936 - val_precision_m: 0.6796 - val_recall_m: 0.7540
time: 16min 20s


Predictions for test_input for the trained model

In [91]:
pred=model.predict(test_input,verbose=1)

1800/1800 [==============================] - 36s 20ms/sample
time: 36 s


Write the predictions for test dataset into answer.txt with columns id and label

In [92]:

sub=pred.round()
sub=le.inverse_transform(sub.ravel().astype('int16'))
sub=pd.DataFrame(sub,columns=['label'])
sub=pd.concat([test['id'], sub], axis=1)
sub.head()
sub.to_csv('answer.txt',sep=',',index=False,header=None)

time: 14.3 ms


Save model weights

In [85]:
# model_save_name = 'bert_weights3.h5'
# path = F"/content/drive/My Drive/BertModelWeights/{model_save_name}" 
# model.save_weights(path)

time: 2.29 s


Loading model weights from already generated saved model weights file after training


In [86]:
model2 = build_model(bert_layer, max_len=max_len)
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
context_word_ids (InputLayer)   [(None, 256)]        0                                            
__________________________________________________________________________________________________
context_mask (InputLayer)       [(None, 256)]        0                                            
__________________________________________________________________________________________________
context_seg_ids (InputLayer)    [(None, 256)]        0                                            
__________________________________________________________________________________________________
reponse_word_ids (InputLayer)   [(None, 256)]        0                                            
____________________________________________________________________________________________

Load model weights

In [87]:
print('download model weights')

#Previously saved models
#bert_model3.h5
#https://drive.google.com/file/d/1cmU8bl_JzywEGegw_3iUFzHrS2iYuwj1/view?usp=sharing


#Download previously saved bert_model3.h5 which used the same training process
!gdown --id 1cmU8bl_JzywEGegw_3iUFzHrS2iYuwj1





download model weights
Downloading...
From: https://drive.google.com/uc?id=1cmU8bl_JzywEGegw_3iUFzHrS2iYuwj1
To: /content/bert_weights3.h5
438MB [00:02, 184MB/s]
time: 5.55 s


Load the weights into model2 from the downloaded file

In [88]:
print('load model')
model2.load_weights('bert_weights3.h5')

load model
time: 452 ms


Predictions for model2 after loading weights

In [89]:
pred2 = model2.predict(test_input,verbose=1)



1800/1800 [==============================] - 39s 22ms/sample
time: 38.8 s


Write the predictions for test dataset into answer.txt with columns id and label

In [90]:
sub=pred2.round()
sub=le.inverse_transform(sub.ravel().astype('int16'))
sub=pd.DataFrame(sub,columns=['label'])
sub=pd.concat([test['id'], sub], axis=1)
sub.head()
sub.to_csv('answer.txt',sep=',',index=False,header=None)

time: 19.8 ms
